## DSC Courses: Assignment Grades Main

In [30]:
import sys
sys.path.insert(0, 'src')

# tables
import numpy as np
import pandas as pd

# basic packages
from glob import glob
import json
import re

# grades, canvas
from canvasapi import Canvas
from third_parties import *

### Setup

#### Canvas Setup

In [31]:
# load credential
CREDENTIAL_FILE = '.info/canvas_credentials.json'
with open(CREDENTIAL_FILE) as f:
    canvas_credential = json.load(f)

In [32]:
# Setup: Canvas
API_URL = "https://canvas.ucsd.edu/"
API_KEY = canvas_credential['API_KEY']        # Generate Access Token from Canvas
COURSE_ID = canvas_credential['COURSE_ID']    # From Canvas Website
COURSE = Canvas(API_URL, API_KEY).get_course(COURSE_ID)

print(COURSE.name) # make sure title and quarter is correct

DSC 20 - Prgrmng/DataStruc for Data Sci - Langlois [FA23]


#### Personnel Setup

In [33]:
# Setup: Student Profiles (ID, Name, Email)
student_ls = []
for stu in COURSE.get_users(enrollment_type=['student'], enrollment_state=['active']):
    student_ls.append([stu.id, stu.name, stu.email])
STUDENTS = pd.DataFrame(student_ls, columns=["id", "name", "email"])
STUDENTS.to_csv(".info/students.csv")
print(STUDENTS.shape[0])

112


In [34]:
# Setup: Staff Profiles
STAFF = pd.read_csv('.info/staff_fa23.csv')['Email'].unique()

In [35]:
# Setup: Mismatched emails
with open('.info/email_records.json') as f:
    EMAIL_RECORDS = json.load(f)
EMAIL_RECORDS

{}

### Slido

In [ ]:
# setup
fname = "lec01.xlsx"  # change file name
aname = 'Lecture 01 (09/23)'  # change for each lecture

# create gradebook
slido_lec = Slido(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname)
gradebook = slido_lec.create_gradebook()
gradebook

In [ ]:
# once inspected, enter grades
slido_lec.enter_grades()

### WebClicker

In [36]:
# setup
fname = "DSC20 Votes.csv"  # change file name
aname = 'Lecture 01 (09/29)'  # change for each lecture

# create gradebook
wc = WebClicker(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname)
gradebook = wc.create_gradebook(session=1, min_poll=0.5) # change session name
gradebook

AttributeError: 'WebClicker' object has no attribute 'use_name'

In [ ]:
# replace values starting with 'X' with NaN
self.gradebook[session_cols] = (
    self.gradebook[session_cols]
    .replace(to_replace = r'X[A-Z]', value = np.nan, regex = True)
)

In [8]:
# once inspected, enter grades
wc.enter_grades()

NameError: name 'wc' is not defined

### Zybook Readings

In [ ]:
# setup
fname = "lec01.csv"  # change file name
aname = 'Zybook Readings 01 (09/23)'  # change for each assignment
due_time = None

# create gradebook
zy_reading = Zybook(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname, due_time=due_time)
config = {
    '10.1': ['Participation', 'Challenge'],
    '10.2': ['Participation', 'Challenge'],
    '10.3': ['Participation', 'Challenge']
}
gradebook = zy_reading.create_gradebook(config=config)
gradebook

In [ ]:
# once inspected, enter grades
zy_reading.enter_grades()

### Discussion Zylabs

In [ ]:
# setup
fname = "disc01.csv"  # change file name
aname = 'Discussion Zylab Week 1 (09/27)'  # change for each assignment
due_time = '2022-11-22 23:59'

# create gradebook
disc_zylab = Zybook(
    COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname,
    dir_name='zylab',
    assignment_group='Discussion Zylab',
    due_time=due_time
)
config = {
    '9.12': ['Lab'],
    '9.13': ['Lab'],
    '13.4': ['Participation'],
    '13.8': ['Lab'],
    '26.21': ['Lab']
}
gradebook = disc_zylab.create_gradebook(config=config)
gradebook

In [ ]:
# once inspected, enter grades
disc_zylab.enter_grades()

### Gradescope Homework/Project

In [ ]:
aname = 'hw01'  # change for each assignment
fname = aname + "_scores.csv"  # main file name
due_time = '2022-01-17 23:59'

# other files
lateness_file = aname + "_lateness.csv"  # lateness file name
redemption_file = aname + "_redemption.csv" # redemption file name
other_section_files = {
    'checkpoint': aname + '_checkpoint.csv',
    # 'quiz': aname + '_quiz.csv'
}

# create gradebook
gs_hw = Gradescope(
    COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname,
    lateness_policy='slip_day',
    lateness_file=lateness_file,
    redemption_file=redemption_file,
    redemption_rate=0.8,
    total_slip_days=5,
    other_section_files=other_section_files,
    due_time=due_time
)

# late_policy = {
#     'no late': 1.0,
#     '1 day': 0.8,
#     '2 days': 0.5
# }

gradebook = gs_hw.create_gradebook()  # defaults take 1 slip day
gradebook

In [ ]:
# once inspected, enter grades
mismatch = gs_hw.enter_grades()
mismatch

In [ ]:
gs_hw.process_slip_day()

### Student Search

In [ ]:
# find student
STUDENTS[STUDENTS['email'].str.lower().str.contains('<email>')]

In [ ]:
# find grade: version 1
gradebook[gradebook.index.str.lower().str.contains('<email>', na=False)]

In [ ]:
# find grade: version 2
gradebook[gradebook['name'].str.lower().str.contains('<name>', na=False)]

### Debug Results

In [ ]:
# modify EMAIL_RECORDS
EMAIL_RECORDS['shreyamusini@gmail.com'] = 'shmusini@ucsd.edu'

In [ ]:
EMAIL_RECORDS

In [ ]:
# DO NOT FORGET to update file
with open('.info/email_records.json', 'w') as f:
    json.dump(EMAIL_RECORDS, f, sort_keys=True)